In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.dilepton_mass import dilepton_mass
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

In [ ]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = True
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'dilepton_mass'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

year = 2018

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
fileset = make_fileset(['DY'], year, redirector=redirector_ucsd, small=False)

# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

#meta = get_sample_meta(fileset, samples)

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress

    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)

    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.dask_executor

if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    print ("I'm running now")

    output = processor.run_uproot_job(
        fileset,
        "Events",
        dilepton_mass(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()
    

In [ ]:
output['totalEvents']['all']/1e6

In [ ]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np


# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import makePlot, scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)
mass_bins       = hist.Bin('mass', r'$mass (GeV/c^2)$', 50, 24, 124)


# define nicer labels and colors

nano_mappings = nano_mapping(year)


my_labels = {
    nano_mappings['TTW'][0]: 'all',
    nano_mappings['TTZ'][0]: 'ttZ',
    nano_mappings['DY'][0]: 'DY',
    nano_mappings['top'][0]: 't/tt+jets',
}

my_colors = {
    nano_mappings['TTW'][0]: '#8AC926',
    nano_mappings['TTZ'][0]: '#FFCA3A',
    nano_mappings['DY'][0]: '#6A4C93',
    nano_mappings['top'][0]: '#1982C4',
}


# 1D Histograms

In [ ]:
from yahist import Hist1D, Hist2D

In [ ]:
tmp1 = output['N_ele'].copy()

tmp2 = output['N_ele2'].copy()

h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('multiplicity').edges()),
)

h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('multiplicity').edges()),
)

fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.plot(ax=ax1, alpha=1, color="C0")
h1.plot(ax=ax1, alpha=1, color="C3")


ax1.set_xlabel(r'$mass\ (GeV/c^2)$')
ax1.set_ylabel(r'Events')

fig.legend(["weighted OS", "SS"])

# Gaussian errors
#    (num/den).plot(ax=ax2,show_errors=True,label="ratio")
# Asymmetric Clopper-Pearson errors
h1.divide(h2, binomial=True).plot(ax=ax2, errors=True, label="OS/SS")


In [ ]:
tmp1 = output['dilep_mass'].copy()
tmp1 = tmp1.rebin('mass', mass_bins)

tmp2 = output['dilep_mass2'].copy()
tmp2 = tmp2.rebin('mass', mass_bins)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('mass').edges()),
)

h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('mass').edges()),
)

fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.plot(ax=ax1, alpha=1, color="C0")
h1.plot(ax=ax1, alpha=1, color="C3")


ax1.set_xlabel(r'$mass\ (GeV)$')
ax1.set_ylabel(r'Events')

fig.legend(["weighted OS", "SS"])

# Gaussian errors
#    (num/den).plot(ax=ax2,show_errors=True,label="ratio")
# Asymmetric Clopper-Pearson errors
h1.divide(h2, binomial=True).plot(ax=ax2, errors=True, label="OS/SS")


In [ ]:
tmp1 = output['dilep_mass'].copy()

tmp1.sum('dataset').values()

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

from Tools.config_helpers import redirector_fnal, redirector_ucsd
from Tools.nano_mapping import make_fileset
from Tools.helpers import get_samples
from Tools.gen import get_charge_parent, find_first_parent

import awkward as ak
import numpy as np


fileset = make_fileset(['DY'], 2018, redirector=redirector_ucsd, small=True)

# load a subset of events
n_max = 500000
events = NanoEventsFactory.from_root(
    fileset[list(fileset.keys())[0]][0],
    schemaclass = NanoAODSchema,
    #entry_stop = n_max,
).events()

In [ ]:
import time

from Tools.objects import Collections

start_time = time.time()
electron = Collections(events, 'Electron', 'tightFCNC', verbose=1).get()
muon = Collections(events, 'Muon', 'tightFCNC', verbose=1).get()

delta_time = time.time()-start_time

print ("\nTook %s seconds"%delta_time)

In [ ]:
from Tools.objects import *

electron = electron[(electron.pt > 20) & (np.abs(electron.eta) < 2.4)]
electron = electron[(electron.genPartIdx >= 0)]
electron = electron[(np.abs(electron.matched_gen.pdgId)==11)]  #from here on all leptons are gen-matched
electron = electron[( (electron.genPartFlav==1) | (electron.genPartFlav==15) )] #and now they are all prompt

muon = muon[(muon.pt > 20) & (np.abs(muon.eta) < 2.4)]
muon = muon[(muon.genPartIdx >= 0)]
muon = muon[(np.abs(muon.matched_gen.pdgId)==13)] #from here, all muons are gen-matched
muon = muon[( (muon.genPartFlav==1) | (muon.genPartFlav==15) )] #and now they are all prompt

lepton   = ak.concatenate([muon, electron], axis=1)
dilepton = choose(lepton, 2)

dilepton = choose(lepton, 2)
dilepton_mass = (dilepton['0']+dilepton['1']).mass

In [ ]:
dilepton_mass

In [ ]:
ak.flatten(dilepton_mass[(ak.num(muon) <= 1)])